In [6]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


## CONFIG

In [13]:
os.getcwd()
external_hardisk_data_path = os.path.abspath(os.sep) + "./Volumes/Lacie/Database/"

train_path = external_hardisk_data_path + "Single_channel_two_patient/train"

test_path = external_hardisk_data_path + "Single_channel_two_patient/test"

test_df = pd.read_csv("single_patient_data/test_data")

resnet152_input_shape = (224, 224, 3)

inceptionV3_input_shape = (299, 299, 3)

batch_size = 1

In [3]:
single_patient_model = tf.keras.models.load_model("h5_files/single.h5")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-10-23 19:06:35.287163: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-23 19:06:35.287546: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
single_patient_model.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

## Get labels

In [8]:
labels = test_df['class'].unique()

In [9]:
labels

array(['Interictal', 'Preictal_One', 'Preictal_Two', 'Seizure'],
      dtype=object)

## Get test windows png's

In [18]:
generator = ImageDataGenerator(
  rescale=1./255
)

In [19]:
resnet_train_gen = generator.flow_from_dataframe(
    dataframe=test_df,
    batch_size=batch_size, 
    target_size=resnet152_input_shape, 
    classes=list(labels), 
    shuffle=False)
inception_train_gen = generator.flow_from_dataframe(
    dataframe=test_df,
    batch_size=batch_size, 
    target_size=inceptionV3_input_shape, 
    classes=list(labels), 
    shuffle=False)

Found 0 validated image filenames belonging to 4 classes.
Found 0 validated image filenames belonging to 4 classes.


/usr/local/Caskroom/miniforge/base/envs/speciale_01_01/lib/python3.9/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 879 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [14]:
def test_generator_two_img():
    while True:
        X1i = inception_train_gen.next()
        X2i = resnet_train_gen.next()
        yield [X1i[0], X2i[0]], X1i[1]

## test model

In [22]:
number_of_steps = len(resnet_train_gen.filenames) // batch_size